In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import init
import torch.autograd as autograd

import torch.nn as nn
import torch.optim as optim
import torchvision

import copy
import numpy as np
import scipy.stats as ss
import scipy
import h5py
import random
import argparse
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pylab as plt

import warnings
from torch.nn.modules.utils import _single, _pair
import math
import copy
# do crazy end-to-end optimization of backward path with MNIST/CIFAR10 and nn.linear    
# when calling loss.backward both gradinet_weight and grad_weight_feedback are being computed in customized modules
# first control against BP and regular modules
# Linear from here :https://pytorch.org/docs/stable/notes/extending.html
# Conv from here: https://github.com/pytorch/pytorch/blob/master/torch/nn/grad.py    
# autograd : https://pytorch.org/docs/stable/autograd.html#torch.autograd.backward  

class ReLUBFunction(torch.autograd.Function):
    """
    ReLUB (ReLU Back) is a nonlinearity interms of input2 which is used in 
    backward path
    """

    @staticmethod
    def forward(ctx, input, input2):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input, input2)
        input[input2<0] = 0
        return input #input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, input2, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input2 < 0] = 0
        return grad_input, None

class ReLUB(nn.Module):
    def __init__(self):
        super(ReLUB, self).__init__()
    
    def forward(self, input, input2):
        return ReLUBFunction.apply(input, input2)#, input2.clamp(min=0).detach()


import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torchvision.models as models
# from modules import customized_modules_simple as customized_modules
from modules import BiHebb_modules as customized_modules  # used for NeuIPS2020

Linear = customized_modules.Linear
ReLU = nn.ReLU() #customized_modules.ReLU

# class FullyConnectedF(nn.Module):
#     def __init__(self, algorithm):
#         super(FullyConnectedF, self).__init__()
#         input_length = 1024
#         self.fc_0 = Linear(input_length,256,False, algorithm=algorithm)
#         self.fc_1 = Linear(256,256,False, algorithm=algorithm)
#         self.fc_2 = Linear(256,10,False, algorithm=algorithm)
#         self.relu = ReLU
       
#     def forward(self,x):
#         x = self.relu(self.fc_0(x))
#         x = self.relu(self.fc_1(x))
#         x = self.relu(self.fc_2(x))
       
#         return x, x # just to satisfy the requirements

          

# class FullyConnectedB(nn.Module):
#     def __init__(self, hidden_layers, nonlinearfunc, input_length, algorithm):
#         super(FullyConnectedB, self).__init__()
#         input_length = 1024
#         self.fc_0 = Linear(256, input_length,False, algorithm=algorithm)
#         self.fc_1 = Linear(256,256,False, algorithm=algorithm)
#         self.fc_2 = Linear(10,256,False, algorithm=algorithm)
#         self.relu = ReLUB
       
#     def forward(self,x):
        
#         x = self.relu(self.fc_2(x))
#         x = self.relu(self.fc_1(x))
#         x = self.relu(self.fc_0(x))
#         return x, x # just to satisfy the requirements (the first one shoudl be V1 layer)
        


In [43]:
import socket
if socket.gethostname()[0:4] in  ['node','holm','wats']:
    path_prefix = '/rigel/issa/users/Tahereh/Research'
elif socket.gethostname() == 'SYNPAI':
    path_prefix = '/hdd6gig/Documents/Research'
elif socket.gethostname()[0:2] == 'ax':
    path_prefix = '/scratch/issa/users/tt2684/Research'
    plt.switch_backend('agg')
elif socket.gethostname() == 'turing':
    path_prefix = '/home/tahereh/Documents/Research'


imagesetdir = path_prefix+'/Data/'
resultsdir = path_prefix+'/Results/Toy_models/'

class Args:
    dum=None

args = Args()
args.batch_size=256
args.no_cuda = False
args.seed = 0

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

use_cuda = True
# data loader
kwargs = {'num_workers': 0, 'pin_memory': True, 'drop_last':True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1, 0.1),scale=(0.25,2) ),
                        transforms.Resize(32),
                        transforms.ToTensor(),
                       
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(imagesetdir, train=False, transform=transforms.Compose([
                        transforms.RandomAffine(degrees=20, translate=(0.1, 0.1),scale=(0.25,2) ),
                        transforms.Resize(32),
                        transforms.ToTensor(),
                        
                    ])),
    batch_size=args.batch_size, shuffle=False, **kwargs)

# transforms.Normalize((0.1307,), (0.3081,))
n_layers = 3
algorithm = args.algorithm #'inverse'#'forward'#'inverse'#'BP'
n_dataloader = 100000
batch_size = args.batch_size


# class ReLUGrad(nn.Module):
#     def __init__(self):
#         super(ReLUGrad, self).__init__()
#     def forward(self, grad_output, input):
#         grad_input = grad_output.clone()
#         grad_input[input < 0] = 0
#         return grad_input

class Forward(nn.Module):
    def __init__(self, algorithm):
        super(Forward, self).__init__()
        self.fc_0 = Linear(1024, 256, bias=False, algorithm=algorithm)
        self.fc_1 = Linear(256, 10, bias=False, algorithm=algorithm)
        self.relu = nn.ReLU()

    def forward(self, x):
        xrelu = self.fc_0(x)
        x0 = self.relu(xrelu)
        x1 = self.fc_1(x0)

        return x1 , [x, x0, x1], xrelu

class Backward(nn.Module):
    def __init__(self, algorithm):
        super(Backward, self).__init__()
        self.fc_1 = Linear(10, 256, bias=False, algorithm=algorithm)
        self.fc_0 = Linear(256, 1024, bias=False, algorithm=algorithm)
        self.grelu = ReLUB()

    def forward(self, x, x0):
        x1 = self.fc_1(x)
        x1 = self.grelu(x1, x0)
        x0 = self.fc_0(x1)

        return x0, [x1,  x]

def transpose_weights(state_dict):

    state_dict_new = {}
    for k, item in state_dict.items():
        state_dict_new.update({k: item.t()})
    return state_dict_new




In [46]:

algorithm = 'FA'
modelF = Forward(algorithm=algorithm).cuda() # main model
modelB = Backward(algorithm=algorithm).cuda() # backward network to compute gradients for modelF

modelC = Forward(algorithm=algorithm).cuda() # Forward Control model to compare to BP
modelC.load_state_dict(modelF.state_dict())
# start symmetric
# modelB.load_state_dict(transpose_weights(modelF.state_dict()) )
modelE = Forward(algorithm=algorithm).cuda()

optimizerC = optim.RMSprop(modelC.parameters(), lr=1e-2, weight_decay=1e-4)
optimizerF = optim.RMSprop([p for n,p in modelF.named_parameters() if 'feedback' not in n],  lr=1e-2, weight_decay=1e-4)
optimizerB = optim.RMSprop([p for n,p in modelB.named_parameters() if 'feedback' not in n],  lr=1e-2, weight_decay=1e-4)


criterionF = nn.CrossEntropyLoss() #
criterionB = nn.MSELoss() #

n_classes = 10
onehot = torch.zeros(train_loader.batch_size, n_classes).cuda()


for epoch in range(10):
    running_loss = 0.0
    acc = 0.0
    for images, targets in train_loader:
        images = images.cuda()
        targets = targets.cuda()

        onehot = torch.zeros(train_loader.batch_size, n_classes).cuda()
        onehot.zero_()
        onehot.scatter_(1, targets.view(train_loader.batch_size,-1), 1)

        images = images.view(images.shape[0], -1)
        out, interm, xrelu = modelF(images)
        loss = criterionF(out, targets)

        
        optimizerF.zero_grad()
        loss.backward()
        optimizerF.step()

        # # SL
        # modelB.load_state_dict(transpose_weights(modelF.state_dict()))
        # out, interm, xrelu = modelF(images)
        # recons, _ = modelB(onehot-out.detach(), xrelu)
        # optimizerB.zero_grad()
        # lossB = criterionB(recons, images)
        # optimizerB.step()
        # modelF.load_state_dict(transpose_weights(modelB.state_dict()))


        running_loss += loss.item()
        _, predicted = torch.max(out.data, 1)
        acc += (predicted == targets).sum().item()/out.shape[0]

    print(epoch, running_loss/len(train_loader), acc/len(train_loader))


0 2.255288193368504 0.7099692841880342
1 1.575207284373096 0.7424879807692307
2 1.8185051129414485 0.7407184829059829
3 1.8765116889252622 0.7203859508547008


KeyboardInterrupt: 

In [ ]:
predicted